# Initialize

In [0]:
#!import "../Utils/ImportCalculationMethods"

## DataSource

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());

# Dimensions

## AmountType

In [0]:
var amountTypes = @"@@AmountType
SystemName,DisplayName,Parent
T,Technical Result,
PR,Premium,T
LO,Loss,T
RA,Risk Adjustment,
CI,Claims Incurred,
CL,Claims,T
NIC,Claims Paid (LIC),CL
CE,Claims Expense,CL
E,Expenses,
DAE,Deferred Acquisition Expenses,E
AEM,Maintenance Expenses,E
CE,Claim Expenses,E
O,Other,
CU,CoverageUnits,O"
;
var ratioAmountTypes = @"@@RatioAmountType
SystemName,DisplayName,Parent
LR,LossRatio,
S2R,Solvency Ratio,"
;
var patternAmountTypes = @"@@PatternAmountType
SystemName,DisplayName,Parent
XC,Claims Incurred Pattern,
XN,Claims Paid Pattern,"
;
var assetAmountTypes = @"@@AssetAmountType
SystemName,DisplayName,Parent
SH,Shares,
RE,Real Estate,
CH,Cash,
BO,Bonds,
CB,CouponBond,BO
CBC,Coupon,CB
CBF,FaceValue,CB"
;
var solvencyBsAmountTypes = @"@@SolvencyBsAmountType
SystemName,DisplayName,Parent
S2A,Solvency II Assets,
S2IA,Invested Assets,S2A
S2L,Solvency II Liabilities,
S2TP,Technical Provisions,S2L
S2RM,Risk Margin,S2TP
S2BEL,Best Estimate Liability,S2TP
S2OF,Own Funds,S2L
S2SCR,SCR,S2OF
S2FC,Free Capital,S2OF"
;
var ifrsBsAmountTypes = @"@@IfrsBsAmountType
SystemName,DisplayName,Parent
IFRSA,IFRS17 Assets,
IFRSIA,Invested Assets,IFRSA
IFRSOCI,Asset OCI,IFRSA
IFRSL,IFRS17 Liabilites,
IFRSBE,Best Estimate Liabilities,IFRSL
IFRSBEL,BEL Book Value,IFRSBE
IFRSBELOCI,Best Estimate Liabilities OCI,IFRSBE
IFRSRA,Risk Adjustment,IFRSL
IFRSCSM,CSM,IFRSL
IFRSEQ,Shareholders Equity,
IFRSEQOCI,Net OCI Reserve,IFRSEQ
IFRSEQRE,Retained Earnings,IFRSEQ"
;

In [0]:
await Import.FromString(amountTypes).WithType<AmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(ratioAmountTypes).WithType<RatioAmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(patternAmountTypes).WithType<PatternAmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(assetAmountTypes).WithType<AssetAmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(solvencyBsAmountTypes).WithType<SolvencyBsAmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(ifrsBsAmountTypes).WithType<IfrsBsAmountType>().WithTarget(DataSource).ExecuteAsync();

In [0]:
(await DataSource.Query<AmountType>().ToArrayAsync()).Count()

## Cashflow Definition
### Basic

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var basicCashflowDefinitions = @"@@BasicCashflowDefinition
DataNode,AmountType,Shape,InitialValue,Length,Shift,Values0,Values1,Values2,Values3,Values4,
G.BBA.LRC.2020.P,PR,Constant,10000,5,0,,,,,
G.BBA.LRC.2020.P,PR,Linear,1000,4,1,0.5,0.5,0.5,0.5,
";

In [0]:
await Import.FromString(basicCashflowDefinitions).WithType<BasicCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<BasicCashflowDefinition>()

### Bounded

In [0]:
var boundedCashflowDefinitions = @"@@BoundedCashflowDefinition
DataNode,AmountType,Shape,InitialValue,Length,Shift,Minimum,Maximum
G.BBA.LRC.2020.P,PR,Random,6000,4,1,100,400
";

In [0]:
await Import.FromString(boundedCashflowDefinitions).WithType<BoundedCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<BoundedCashflowDefinition>()

### Referenced

In [0]:
var referencedCashflowDefinition = @"@@ReferencedCashflowDefinition
DataNode,AmountType,AccidentYear,Shape,InitialValue,Length,Shift,ReferenceAmountType,Values0,Values1,Values2,Values3,Values4
G.BBA.LRC.2020.P,PR,,Reference,,,,PR,0.5,0.5,0.5,0.5,0.5
G.BBA.LRC.2020.P,CL,,Reference,,,,PR,0.25,0.25,0.25,0.25,0.25
G.BBA.LRC.2020.P,CL,,Reference,,,,AEM,0.5,0.5,0.5,0.5,0.5
G.BBA.LRC.2020.P,AEM,,Reference,,,,PR,0.5,0.5,0.5,0.5,0.5
";

In [0]:
await Import.FromString(referencedCashflowDefinition).WithType<ReferencedCashflowDefinition>().WithTarget(Workspace).ExecuteAsync()

In [0]:
Workspace.Query<ReferencedCashflowDefinition>()